In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tqdm import tqdm
from utils import get_file_path, detect_objects_in_image, count_people_in_image, draw_boxes_on_image, set_style, plot_distribution

# Set Matplotlib style
set_style()

# Set seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Set dataset path
LABELS_PATH = 'dataset/labels.csv'
DATASET_PATH = 'dataset/images.npy'

2023-07-11 22:50:55.043092: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-11 22:50:55.450427: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:7704] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-07-11 22:50:55.450463: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-07-11 22:50:55.451601: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1520] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-11 22:50:55.661188: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-11 22:50:55.664024: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
# Load dataset
labels = pd.read_csv(LABELS_PATH)
labels['file_path'] = labels['id'].apply(get_file_path)

labels.head()

counts = labels['count'].values

In [3]:
# loading the images in vector format
dataset = np.load(DATASET_PATH, allow_pickle=True)

In [4]:
dataset.shape

(2000, 480, 640, 3)

In [5]:
x_train, x_test, y_train, y_test = train_test_split(
    dataset,
    counts,
    test_size=0.2,
    random_state=42
)

In [6]:
# define a LeNet-5 model

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        activation='relu',
        input_shape=(480, 640, 3)
    ),
    tf.keras.layers.MaxPooling2D(
        pool_size=(2, 2)
    ),
    tf.keras.layers.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        activation='relu'
    ),
    tf.keras.layers.MaxPooling2D(
        pool_size=(2, 2)
    ),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(
        units=128,
        activation='relu'
    ),
    tf.keras.layers.Dense(
        units=1,
        activation='linear'
    )
])

model.compile(
    loss='mse',
    optimizer='adam',
    metrics=['mae']
)



2023-07-11 22:51:03.025689: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1221853184 exceeds 10% of free system memory.
2023-07-11 22:51:03.576617: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1221853184 exceeds 10% of free system memory.
2023-07-11 22:51:03.756348: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1221853184 exceeds 10% of free system memory.


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 478, 638, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 239, 319, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 237, 317, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 118, 158, 128)     0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 2386432)           0         
                                                                 
 dense (Dense)               (None, 128)               3

In [8]:
# define a callback to stop training when the validation loss is not decreasing anymore
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# define a callback to save the model with the best validation loss
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'models/model.h5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    save_freq='epoch'
)

# define a callback to reduce the learning rate when the validation loss is not decreasing anymore
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=5,
    verbose=1,
    mode='min'
)

In [9]:
# train the model
history = model.fit(
    x=x_train,
    y=y_train,
    validation_data=(x_test, y_test),
    batch_size=32,
    epochs=100,
    callbacks=[early_stopping, checkpoint, reduce_lr]
)

Epoch 1/100


2023-07-11 22:51:05.677279: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1474560000 exceeds 10% of free system memory.
2023-07-11 22:51:06.766546: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1221853184 exceeds 10% of free system memory.


: 

: 

In [ ]:
# save the training history
pd.DataFrame(history.history).to_csv('results/history_lenet.csv', index=False)

# save the model
model.save('models/model_100.h5')

In [ ]:
# plot the training history
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

ax[0].plot(history.history['loss'], label='Training Loss', lw=3)
ax[0].plot(history.history['val_loss'], label='Validation Loss', linestyle='--', lw=3)

ax[1].plot(history.history['mae'], label='Training MAE', lw=3)
ax[1].plot(history.history['val_mae'], label='Validation MAE', linestyle='--', lw=3)

ax[0].set_xlabel('Epochs', fontsize=15)
ax[1].set_xlabel('Epochs', fontsize=15)

ax[0].set_ylabel('Loss', fontsize=15)
ax[1].set_ylabel('MAE', fontsize=15)

ax[0].legend(fontsize=15)
ax[1].legend(fontsize=15)

ax[0].set_title('Training and Validation Loss', fontsize=15)
ax[1].set_title('Training and Validation MAE', fontsize=15)

plt.savefig('results/history_lenet.png', dpi=300, bbox_inches='tight')

In [ ]:
prediction = pd.DataFrame({
    'y_true': y_test,
    'y_pred': model.predict(x_test).flatten()
})

prediction.head()

In [ ]:
# plot the prediction
